In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.regression.linear_model import yule_walker
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit



df = pd.read_csv(f"../data/input_data/MAIN_DATASET.csv", index_col=[0])
df['year'] = pd.DatetimeIndex(df['date_time']).year
df['month'] = pd.DatetimeIndex(df['date_time']).month
selceted_df = df[df['year'] == 2021]
selceted_df = selceted_df[selceted_df['month'] >= 6]

day_ahead_prices = selceted_df[["dato_id","NO2_price", "NO5_price"]]


In [ ]:


time_steps = 3
order = 3



train = day_ahead_prices["NO2_price"][:-time_steps].to_numpy()
test = day_ahead_prices["NO2_price"][-time_steps:].to_numpy()


def AR_model(train:np.ndarray, test:np.ndarray, time_steps:int,model_order:int, is_differenced = False)->np.ndarray:
    phi, _ = yule_walker(train, order = order)
    #create empty array with lagged values and room for preds(currently zeros)
    X_hat = np.zeros(model_order + time_steps)
    X_hat[:order] = train[-order:]
    
    
    print(f"{X_hat=}")
    # print(f"{phi=}")

    for step in range(time_steps):
        X_t = 0
        for p in range(model_order):
            print(f"step: [{step}], p:[{p}]  {X_hat[(model_order+step)-(p+1)]}")
            X_t += phi[p] * X_hat[(model_order+step)-(p+1)]
        X_hat[model_order+step] = X_t
    print(f"{X_hat[-time_steps:]=}")
    print(f"{test=}")
    
    
AR_model(train,test,time_steps,order)
# for t in range(time_steps):
#     X_t = 0
#     for i in range(1,len(coefficients)+1):
#         X_t += coefficients[i-1] * X_hat[-i]
    
#     X_hat[-t] = X_t
  

# # print(X_hat[-time_steps:])
# # print(test)
# print(f"{mean_squared_error(test,X_hat[-time_steps:])=}")
# plt.plot(np.arange((time_steps), X_hat[-time_steps:]))
# # plt.plot(np.arange((test), test))
# print()
# print('Test R2:', r2_score(validation.iloc[-10:, 0], validation.iloc[-10:, 1]))

# # Plot the newly obtained list
# plot_window = n_steps+24

# plt.plot(np.arange(plot_window),validation["Prediction"].tail(plot_window))
# plt.plot(np.arange(plot_window),validation["Original"].tail(plot_window))
# plt.legend(validation.columns)
# plt.title(f"{n_steps} hour prediction with AR({order})")
# plt.ylabel('Day Ahead Price')
# plt.xlabel('t')
# plt.show()